In [1]:
# Install OpenCV (headless build, no GUI)
!pip install --quiet opencv-python-headless

In [5]:

import cv2
import numpy as np
import os

In [6]:
class SkinColorModel:
    def __init__(self, bins=32):
        self.bins = bins
        self.skin_hist = np.zeros((bins, bins, bins), dtype=np.float64)
        self.non_skin_hist = np.zeros((bins, bins, bins), dtype=np.float64)
        self.bin_size = 256 // bins
        self.skin_count = 0
        self.non_skin_count = 0

    def add_skin_sample(self, image):
        for pixel in image.reshape(-1, 3):
            if not np.array_equal(pixel, [255, 255, 255]):
                r, g, b = pixel
                r_bin, g_bin, b_bin = r // self.bin_size, g // self.bin_size, b // self.bin_size
                self.skin_hist[r_bin, g_bin, b_bin] += 1
                self.skin_count += 1

    def add_non_skin_sample(self, image):
        for pixel in image.reshape(-1, 3):
            r, g, b = pixel
            r_bin, g_bin, b_bin = r // self.bin_size, g // self.bin_size, b // self.bin_size
            self.non_skin_hist[r_bin, g_bin, b_bin] += 1
            self.non_skin_count += 1

    def compute_probabilities(self):
        epsilon = 1e-10
        self.skin_prob = self.skin_hist / (self.skin_count + epsilon)
        self.non_skin_prob = self.non_skin_hist / (self.non_skin_count + epsilon)

    def classify_image(self, image, threshold=1.0):
        h, w, _ = image.shape
        output = np.ones_like(image) * 255

        for i in range(h):
            for j in range(w):
                r, g, b = image[i, j]
                r_bin, g_bin, b_bin = r // self.bin_size, g // self.bin_size, b // self.bin_size
                skin_p = self.skin_prob[r_bin, g_bin, b_bin]
                non_skin_p = self.non_skin_prob[r_bin, g_bin, b_bin]
                ratio = skin_p / (non_skin_p + 1e-10)

                if ratio > threshold:
                    output[i, j] = image[i, j]

        return output


In [7]:
def load_images_from_dir(directory):
    return [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]


In [8]:
def main():
    bins = 32
    threshold = 1.0
    model = SkinColorModel(bins)

    # === Setup paths ===
    train_dir = "train_skin"         # 3 images with skin pixels only (rest white)
    test_dir = "test"                # 2 original test images
    output_dir = "results"           # Save results here

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("[INFO] Loading training images...")
    train_images = load_images_from_dir(train_dir)

    for img_path in train_images:
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARNING] Could not read image: {img_path}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        model.add_skin_sample(img_rgb)

        # Create non-skin version by extracting only white regions
        mask = np.all(img_rgb == [255, 255, 255], axis=-1)      # shape (H, W)
        mask_3d = np.stack([mask] * 3, axis=-1)                 # shape (H, W, 3)
        non_skin_only = np.where(mask_3d, img_rgb, [255, 255, 255])  # keep only white pixels

        model.add_non_skin_sample(non_skin_only)

    print("[INFO] Computing probability histograms...")
    model.compute_probabilities()

    print("[INFO] Processing test images...")
    test_images = load_images_from_dir(test_dir)

    for img_path in test_images:
        filename = os.path.basename(img_path)
        img = cv2.imread(img_path)
        if img is None:
            print(f"[WARNING] Could not read image: {img_path}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        output_img = model.classify_image(img_rgb, threshold)
        result_bgr = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)

        save_path = os.path.join(output_dir, f"result_{filename}")
        cv2.imwrite(save_path, result_bgr)
        print(f"[INFO] Saved result: {save_path}")

    print("[INFO] Skin detection complete.")

if __name__ == "__main__":
    main()


[INFO] Loading training images...
[INFO] Computing probability histograms...
[INFO] Processing test images...
[INFO] Saved result: results\result_test1.jpg
[INFO] Saved result: results\result_test2.jpg
[INFO] Skin detection complete.
